# Interfaces which produce results for multiple matrices

This notebook explain one way to handle an interface that produces results for multiple matrices, where the classic `Interface` pattern (generator which supports `__next__`) won't work. It assumes familiarity with the Brightway 2.5 concepts described in other notebooks.

There is one obvious generic approach to handle this division of responsibilities, namely:

* The interface to an external system calls for new data
* Proxy objects feed that data into matrices

However, the implementation details can vary. Below is just the first path I thought of, it isn't the only one, or the most correct. My emphasis here was on simplicity, balanced with not adding any extra work (i.e. the external interface should "just work").

In [1]:
%matplotlib inline

In [4]:
import bw2data as bd
import bw2calc as bc
import bw_processing as bwp
import numpy as np
import seaborn as sb
from scipy import stats
from matplotlib import pyplot as plt
from functools import partial

This assumes you have this project, with the ecoinvent database installed.

In [3]:
bd.projects.set_current("ecoinvent 3.7.1")

# Basic case study setup

Ac activity with one technosphere and one biopshere input

In [21]:
if "multiple matrices case study" in bd.databases:
    del bd.databases["multiple matrices case study"]

In [22]:
db = bd.Database("multiple matrices case study")
db.register()

In [23]:
act = db.new_activity(code="foo", name="example activity")
act.save()

In [24]:
co2 = next(x for x in bd.Database("biosphere3") 
           if x['name'] == 'Carbon dioxide, fossil'
           and x['categories'] == ('air',))

In [25]:
steel = next(x for x in bd.Database("ecoinvent 3.7.1") 
             if x['name'] == 'market for steel, low-alloyed')

In [26]:
act.new_exchange(input=co2, amount=1, type="biosphere").save()
act.new_exchange(input=steel, amount=1, type="technosphere").save()
act.new_exchange(input=act, amount=1, type="production").save()

# Interface to external data

In [66]:
class Interface:
    """An interface that gathers data for both technosphere and biosphere matrix elements."""
    def __init__(self):
        self.state = {
            'technosphere': False,
            'biosphere': False
        }        
        self.technosphere_index = 1
        
    def technosphere(self):
        if not self.state['technosphere']:
            self.regenerate()
        self.state['technosphere'] = False
        # Make sure to get sign right here (inputs are consumed -> negative), or in `flip_array`
        return self.data[:self.technosphere_index] * -1

    def biosphere(self):
        if not self.state['biosphere']:
            self.regenerate()
        self.state['biosphere'] = False
        return self.data[self.technosphere_index:]
    
    def regenerate(self):
        # Do whatever magic here
        print("Getting new data")
        self.data = np.random.random(size=2)
        self.state = {
            'technosphere': True,
            'biosphere': True
        }        
        

In [67]:
my_interface = Interface()

In [68]:
my_interface.technosphere()

Getting new data


array([-0.73817437])

In [69]:
my_interface.data

array([0.73817437, 0.75994679])

Asking for `biosphere` uses the saved data:

In [70]:
my_interface.biosphere()

array([0.75994679])

But asking for either again draws new data:

In [71]:
my_interface.technosphere()

Getting new data


array([-0.13109311])

In [72]:
my_interface.data

array([0.13109311, 0.25726256])

# Proxy classes

In [73]:
class TechnosphereInterface:
    def __init__(self, interface):
        self.interface = interface
    
    def __next__(self):
        return self.interface.technosphere()

In [74]:
class BiosphereInterface:
    def __init__(self, interface):
        self.interface = interface
    
    def __next__(self):
        return self.interface.biosphere()

# Create the data package

This will need two resources, as we are interacting with two matrices.

In [75]:
t_indices=np.array([
        (steel.id, act.id), 
    ],
    dtype=bwp.INDICES_DTYPE   
)
b_indices=np.array([
        (co2.id, act.id), 
    ],
    dtype=bwp.INDICES_DTYPE   
)
flip_array=np.array([False])

In [76]:
dp = bwp.create_datapackage()

In [77]:
dp.add_dynamic_vector(
    matrix="technosphere_matrix",
    interface=TechnosphereInterface(my_interface),
    indices_array=t_indices,
    flip_array=flip_array,
)

In [78]:
dp.add_dynamic_vector(
    matrix="biosphere_matrix",
    interface=BiosphereInterface(my_interface),
    indices_array=b_indices,
    flip_array=flip_array,
)

# Use in LCA

In [79]:
ipcc = ('IPCC 2013', 'climate change', 'GWP 100a')

In [80]:
fu, data_objs, _ = bd.prepare_lca_inputs({act: 1}, method=ipcc)

In [81]:
lca = bc.LCA(fu, data_objs=data_objs + [dp])
lca.lci()
lca.lcia()
print(lca.score)

for _ in range(10):
    next(lca)
    print(lca.score)

Getting new data
1.2793434591334893
Getting new data
1.4096416474356124
Getting new data
1.3660240337707836
Getting new data
0.9613782949753171
Getting new data
1.6996463906672228
Getting new data
0.36225888230578684
Getting new data
1.2209777556907158
Getting new data
1.3054816978948893
Getting new data
1.0009585745288458
Getting new data
0.8829430542975789
Getting new data
0.5993998405288866
